In [ ]:
#computes cosine similarities between word embeddings to find most similaire books 
#tfidf pour regler le probleme des grande valeur pour le mot qui ne sont pas important
#on a utilisé l'autre car cette methode n'a pas donnée les meilleur resultat 


In [ ]:
!pip install kaggle
!pip install nltk

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"abdramdani","key":"0850d17ece69fcc4d3fdbf3d1fa789d4"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#data set api 
!kaggle datasets download -d ruchi798/bookcrossing-dataset

 80% 61.0M/76.1M [00:00<00:00, 77.9MB/s]
100% 76.1M/76.1M [00:00<00:00, 120MB/s] 


In [ ]:
#code to extract bookcrossing-dataset.zip
from zipfile import ZipFile
file_name="bookcrossing-dataset.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print("done")

done


In [ ]:
#import 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
books_data=pd.read_csv("/content/Books Data with Category Language and Summary/Preprocessed_data.csv",low_memory=False)
books_rat=pd.read_csv("/content/Book reviews/Book reviews/BX-Book-Ratings.csv",sep=';', encoding='latin-1',low_memory=False)

books_data=books_data.rename(columns={"isbn":"ISBN"})
print(books_data.shape,"\n",list(books_data.columns))

drop_c=['user_id','Unnamed: 0','age', 'location' , 'city', 'state', 'country']
books_data=books_data.drop(drop_c,axis=1)

books_data=books_data.drop_duplicates(subset="ISBN")
#on enleve les column avec l'erreur de 9 
books_data=books_data[books_data["Summary"]!="9"]
books_data=books_data[books_data["Category"]!="9"]
books_data=books_data.drop_duplicates(subset="Summary")
books_data=books_data[books_data["Language"]=="en"]
print(books_data.shape)

(1031175, 19) 
 ['Unnamed: 0', 'user_id', 'location', 'age', 'ISBN', 'rating', 'book_title', 'book_author', 'year_of_publication', 'publisher', 'img_s', 'img_m', 'img_l', 'Summary', 'Language', 'Category', 'city', 'state', 'country']
(130024, 12)


In [ ]:
#jajoute le average rating et la somme des ratting pour les livres
#jenleve les rating =0 pour le calcule 
print("dataset des rating avant : ",books_rat.shape,books_rat["ISBN"].nunique(),books_rat["User-ID"].nunique())
books_rat_nnul=books_rat[books_rat["Book-Rating"]!=0]
print("dataset des rating apres l'elimination des valeurs ==0 : ",books_rat_nnul.shape,books_rat_nnul["ISBN"].nunique(),books_rat_nnul["User-ID"].nunique())


#average_rating
average_book=pd.DataFrame(books_rat_nnul.groupby("ISBN")["Book-Rating"].mean()).reset_index().rename(columns={"Book-Rating":"average_rating"})
#average_book.sort_values(by="average_rating",ascending=False)

books_data=pd.merge(books_data,average_book, on ="ISBN")

#somme rating
book_rating_Count=(books_rat_nnul.groupby(by=["ISBN"])["Book-Rating"].count().reset_index().
                   rename(columns={"Book-Rating":"SommeRating"}))
books_data=pd.merge(books_data,book_rating_Count, on ="ISBN")
print(books_data.shape)
#books_data

dataset des rating avant :  (1149780, 3) 340556 105283
dataset des rating apres l'elimination des valeurs ==0 :  (433671, 3) 185973 77805
(75313, 14)


In [ ]:
#j'utilise que les rating sans les 0
books_rating=pd.merge(books_rat_nnul,books_data["ISBN"],on="ISBN")
print(books_rating.shape,books_rating.columns)

(216994, 3) Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')


In [ ]:
#on prend un user avec > 15 ratting 
print(books_rating.shape,books_rating["User-ID"].nunique(),books_rating["ISBN"].nunique())
#groupe1= books_rat_nnul["User-ID"].value_counts()
groupe1=pd.DataFrame(books_rating.groupby("User-ID")["ISBN"].count()).reset_index().rename(columns={"ISBN":"Nb_book_rated"})
print(groupe1.shape)
groupe1=groupe1.query("Nb_book_rated >=15")
print(groupe1.shape)
books_rat2=pd.merge(books_rating,groupe1["User-ID"], on ="User-ID",how="inner")
print(books_rat2.shape,books_rat2["User-ID"].nunique(),books_rat2["ISBN"].nunique())

(216994, 3) 46174 75313
(46174, 2)
(2563, 2)
(118186, 3) 2563 55049


In [ ]:
#je prends les livres qui existe dans books_rat2
books_rat_isbn=books_rat2.drop_duplicates(subset="ISBN")
print(books_rat_isbn.shape)
books_data2=pd.merge(books_data,books_rat_isbn["ISBN"], on ="ISBN",how="inner")
print(books_data2.shape,books_rat2["ISBN"].nunique())



(55049, 3)
(55049, 14) 55049


In [ ]:
print(books_data2.shape,books_rat2["User-ID"].nunique(),books_rat2["ISBN"].nunique())

(55049, 14) 2563 55049


In [ ]:
"""#or import the readdy dataset 
import pickle

pickle_in=open("/content/books_data_final.pickle","rb")
books_data2=pickle.load(pickle_in)"""

In [ ]:
#importation de nltk 
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer #

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet') #
stop_words = stopwords.words('english')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
#tokenizatoin 
def tok(text):
  return word_tokenize(str(text))
print(tok("Hello World "))


['Hello', 'World']


In [ ]:
#on enleve les ponctuation 
def enleve_ponct(text):
  #symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
  symbols = "!\"0123456789#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
  for i in range(len(symbols)):
      text = np.char.replace(text, symbols[i], ' ')
      text = np.char.replace(text, "  ", " ")
  text = np.char.replace(text, ',', '')
  return str(text)


In [ ]:
# le text en miniscule 
def miniscule(text):
  #return str(np.char.lower(text))
  return text.lower() 


In [ ]:
#stop words
def remove_stop_words(text):
  #stop_words = stopwords.words('english')
  words = word_tokenize(str(text))
  new_text = ""
  for w in words:
    if w not in stop_words and len(w) > 1:
      new_text = new_text + " " + w
  return new_text


In [ ]:
#Stemming connection connected ... -> connect 
def stemm(text):
  ps=PorterStemmer()
  new_text=""
  words=word_tokenize(str(text))
  for word in words:
    new_text=new_text+" "+ps.stem(word)
  return new_text


In [ ]:
#Lemmatization
def lemm(text):
  lem = WordNetLemmatizer()
  new_text=""
  words=word_tokenize(str(text))
  for word in words:
    new_text=new_text+" "+lem.lemmatize(word,"v")
  return new_text


In [ ]:
def preprocess(row):
  text=row["Summary"]
  text=enleve_ponct(text)
  text=miniscule(text)
  #text=stemm(text)
  text=remove_stop_words(text)
  text=tok(text)
  #text=stemm(text)
  #text=lemm(text)

  return text

In [ ]:
books_test=books_data2

In [ ]:
books_test["Text"]=books_test.apply(preprocess,axis=1) # elle prend +2mn 
#donc je vais directement ajouter cette column a mon dataset plus tard 




In [ ]:
books_test.shape

(55049, 15)

In [ ]:
#!pip install gensim
!pip install --upgrade gensim #pour la version 4.0.1


     |████████████████████████████████| 23.9MB 166kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import gensim
print(gensim.__version__)


4.0.1


/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [ ]:
import gensim.downloader as api 
model = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
sentence =[x for x in books_test["Text"]]
#sentence = books_test["Text"].tolist()

In [ ]:
from gensim.corpora import Dictionary #bagofword
dictionary = Dictionary(sentence) #to fit dictionary


In [ ]:
def dic2boww(row , dictionary=dictionary):
  return dictionary.doc2bow(row["Text"])

In [ ]:
books_test["Text"]=books_test.apply(dic2boww,axis=1) #to convert every sentence to bag of word format

In [ ]:
from gensim.models import TfidfModel
documents = [x for x in books_test["Text"]]

tfidf = TfidfModel(documents)

In [ ]:
def tfidfff(row,tfidf=tfidf):
  return tfidf[row["Text"]]

In [ ]:
books_test["Text"]=books_test.apply(tfidfff,axis=1)
#A term similarity index that computes cosine similarities between word embeddings
#on calcule le cosinus entre les wordembedding
#trouver les word embedding les plus similaire 
#

In [ ]:
dd=books_test["Text"].tolist()

In [ ]:
from gensim.similarities import SparseTermSimilarityMatrix, WordEmbeddingSimilarityIndex
termsim_index = WordEmbeddingSimilarityIndex(model)
termsim_matrix = SparseTermSimilarityMatrix(termsim_index, dictionary, tfidf) #elle prend +4hr(4h12mn) mais sans le prob de la ram 5.13/12.69 utilisé

100%|██████████| 55679/55679 [3:47:24<00:00,  4.08it/s]


In [ ]:
from gensim.similarities import SoftCosineSimilarity #le calcule entre les vecteur avec le cosinus
docsim_index = SoftCosineSimilarity(dd, termsim_matrix, num_best=10) #num_best (1+nombre de livre a recommandé)

In [ ]:
#on prends un livre 
x=np.random.randint(55049)
test=books_test.iloc[x]
sim=docsim_index[test["Text"]]
col=("ISBN","rat","sim")
array = np.empty((0,3))
for i in range(0,10):
  row=books_test.iloc[sim[i][0]]
  array = np.append(array, np.array([[row["ISBN"],row["average_rating"],sim[i][1]]]), axis=0)
df = pd.DataFrame(data=array, columns=col)
#le premier c notre livre 
df.sort_values("sim",ascending=False)


/usr/local/lib/python3.7/dist-packages/gensim/similarities/termsim.py:359: RuntimeWarning: divide by zero encountered in true_divide
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)
/usr/local/lib/python3.7/dist-packages/gensim/similarities/termsim.py:359: RuntimeWarning: invalid value encountered in multiply
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)


,ISBN,rat,sim
0,1893896323,2.0,1.0
1,0786860987,8.25,0.6360043883323669
2,0743423593,6.0,0.6348553895950317
3,0380973324,8.0,0.514671266078949
4,0843948817,8.0,0.40869927406311035
5,0684855283,8.0,0.40869924426078796
6,0425120430,6.8,0.3979695439338684
7,0425192113,9.5,0.3970581293106079
8,0688131379,9.0,0.3970315754413605
9,0425151468,5.0,0.39414432644844055


In [ ]:
"""
realnote=test["average_rating"]
note=0
for i,row in aa.iloc[1:].iterrows():
  note=note+float(row["rat"])
print("real note:",realnote," la note estimé",note/9)

"""

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
test_data=books_test[0:5000]


In [ ]:
def sim_valeur():


  col2=("ISBN","note")
  col=("ISBN","rat")
  arr = np.empty((0,2))
  for i,row in test_data.iterrows():
    sim=docsim_index[row["Text"]]
    array = np.empty((0,2))
    for i in range(0,10):
      roww=books_test.iloc[sim[i][0]]
      array = np.append(array, np.array([[roww["ISBN"],roww["average_rating"]]]), axis=0)
    dff = pd.DataFrame(data=array, columns=col)
    note=0
    for i,roww in dff.iloc[1:].iterrows():
      note=note+float(roww["rat"])
    note=note/9
    arr = np.append(arr, np.array([[row["ISBN"],note]]), axis=0)


  df = pd.DataFrame(data=arr, columns=col2)
  return df


In [ ]:
a=sim_valeur()

In [ ]:
#tfidf
from sklearn.metrics import mean_absolute_error,mean_squared_error
mean_absolute_error(test_data["average_rating"], a["note"])


In [ ]:
mean_squared_error(test_data["average_rating"], a["note"])

0.9485803134356411

In [ ]:
"""pickle_out=open("SoftCosineSimilarityWET2.pickle","wb")
pickle.dump(docsim_index,pickle_out)
pickle_out.close()"""

In [ ]:
"""pickle_out=open("SparseTermSimilarityMatrixWET2.pickle","wb")
pickle.dump(termsim_matrix,pickle_out)
pickle_out.close()"""